In [1]:
import os
import numpy as np
import nibabel

data_path = '/home/roshansk/Covid/3dSeg/Liver/Data/OrigData/'
#we will undersample our training 2D images later (for memory and speed)
image_rows = int(512/2)
image_cols = int(512/2) 

### NPY Method

In [17]:
def create_train_data(train_data_path, masks, imgs, subset):
    print('-'*30)
    print('Creating training data...')
    print('-'*30)
    
    masks_train = []
    imgs_train = []
        
    for liver, orig in zip(masks, imgs):
        #we load 3D training mask (shape=(512,512,129))
        training_mask = nibabel.load(os.path.join(train_data_path, liver))
        #we load 3D training image
        training_image = nibabel.load(os.path.join(train_data_path, orig)) 
        
        for k in range(training_mask.shape[2]):
            #axial cuts are made along the z axis with undersampling
            mask_2d = np.array(training_mask.get_data()[::2, ::2, k]) 
            image_2d = np.array(training_image.get_data()[::2, ::2, k])
            #we only recover the 2D sections containing the liver
            #if mask_2d contains only 0, it means that there is no liver
            if len(np.unique(mask_2d)) != 1:
                masks_train.append(mask_2d)
                imgs_train.append(image_2d)
                    
    imgs = np.ndarray(
            (len(imgs_train), image_rows, image_cols), dtype=np.uint8
            )
    imgs_mask = np.ndarray(
            (len(masks_train), image_rows, image_cols), dtype=np.uint8
            )
    
    for index, img in enumerate(imgs_train):
        imgs[index, :, :] = img
        
    for index, img in enumerate(masks_train):
        imgs_mask[index, :, :] = img

    np.save(f'imgs_{subset}.npy', imgs)
    np.save(f'masks_{subset}.npy', imgs_mask)
    print('Saving to .npy files done.')

In [18]:

imgFiles = sorted([x for x in os.listdir(data_path) if 'orig' in x])
maskFiles = sorted([x for x in os.listdir(data_path) if 'liver' in x])

## Performing Train-Test split
train_img = imgFiles[:16]
train_mask = maskFiles[:16]

test_img = imgFiles[16:]
test_mask = maskFiles[16:]

In [19]:
create_train_data(data_path, train_mask, train_img, 'train')

------------------------------
Creating training data...
------------------------------
Saving to .npy files done.


In [20]:
create_train_data(data_path, test_mask, test_img, 'test')

------------------------------
Creating training data...
------------------------------
Saving to .npy files done.
